In [32]:
import os as os
import pandas as pd
import re as re
os.getcwd()
os.listdir()

['.ipynb_checkpoints',
 'BIOS274-2019-PS2-description.pdf',
 'ENCFF016CBS.tsv',
 'example_URL_code.ipynb',
 'output_file',
 'Problem_Set2_Part1_Xiaotao_Shen.ipynb',
 'ps2_gencode_url.tsv',
 'ps2_gene_quant_URLs.tsv',
 'Untitled.ipynb',
 'Xiaotao Shen code for problem set2.ipynb']

## Read the ps2_gene_quant_URLs.tsv

In [33]:
ps2_gene_quant_url = pd.read_csv("ps2_gene_quant_URLs.tsv", sep='\t', header = None)
ps2_gene_quant_url

,0,1,2
0,ENCFF016CBS,omental fat pad,https://www.encodeproject.org/files/ENCFF016CB...
1,ENCFF365ZMW,sigmoid colon,https://www.encodeproject.org/files/ENCFF365ZM...
2,ENCFF408GFZ,subcutaneous adipose tissue,https://www.encodeproject.org/files/ENCFF408GF...
3,ENCFF505TUS,prostate gland,https://www.encodeproject.org/files/ENCFF505TU...
4,ENCFF633OSJ,suprapubic skin,https://www.encodeproject.org/files/ENCFF633OS...
5,ENCFF862LZL,heart left ventricle,https://www.encodeproject.org/files/ENCFF862LZ...
6,ENCFF863ERP,testis,https://www.encodeproject.org/files/ENCFF863ER...
7,ENCFF916ODF,vagina,https://www.encodeproject.org/files/ENCFF916OD...
8,ENCFF918KPC,stomach,https://www.encodeproject.org/files/ENCFF918KP...


## Creat new folder to output files

In [17]:
os.makedirs("output_file")
os.chdir("output_file")

In [ ]:
i = 0
info = list(ps2_gene_quant_url.iloc[i])
info
info[2]
data = pd.read_csv(info[2], sep='\t')
data
data.to_csv("data.csv")

In [26]:
for i in range(ps2_gene_quant_url.shape[0]):
    print(i, end=" ")
    info = list(ps2_gene_quant_url.iloc[i])
    #read data
    data = pd.read_csv(info[2], sep='\t')
    #only remain 'gene_id', 'TPM'
    data = data[['gene_id', 'TPM']]
    #filter data according to TPM
    data = data[data.TPM > 0.5]
    ##add a new column named file_accession
    data['file_accession'] = info[0]
    ##filter data according to gene_ID
    gene_id = list(data.gene_id)
    remain_idx = [idx for idx in range(len(gene_id)) if gene_id[idx].startswith('ENSG')]
    idx1 = set(list(range(len(gene_id))))
    idx2 = set(remain_idx)
    other_gene_id = data.iloc[list(idx1.difference(idx2))].gene_id
    other_gene_id = pd.DataFrame(other_gene_id)
    data = data.iloc[remain_idx]
    data.gene_id = [re.sub('\\.[0-9]*', '', id) for id in list(data.gene_id)]
    ## output informaiton and gene name which are not start with ‘ENSG’ for each quantification file
    quan_file_name = info[0] + '_quan_file.tsv'
    quan_file_other_gene_name = info[0] + '_other_gene_name.tsv'
    other_gene_id.to_csv(quan_file_other_gene_name, sep='\t', index=False)
    data.to_csv(quan_file_name, sep='\t',index=False)

0 1 2 3 4 5 6 7 8 

## Read the GTF file

In [8]:
os.chdir('..')
print(os.getcwd())
ps2_gencode_url = pd.read_csv("ps2_gencode_url.tsv", sep = "\t", header = None)
print(ps2_gencode_url)

D:\my github\python_class\Problem_Set2
                                                   0
0  https://www.encodeproject.org/files/gencode.v2...


In [9]:
import urllib.request
import gzip

In [10]:
# variable for URL
request_URL = list(ps2_gencode_url.iloc[0])[0]

In [11]:
# open stream for data
response = urllib.request.urlopen(request_URL)

In [12]:
# open gzipped connection
import re as re
gene_info = []
with gzip.open(response, 'rt') as inFile:
    for line in inFile:
        line = line.rstrip('\n')
        # print(line)
        if not line.startswith("#"):
            line = line.split("\t")
            # print(line)
            if line[1] == "HAVANA" and line[2] == "gene":
                # gene.append(line)
                attribute = line[8].split(';')
                gene_id = [i for i in attribute if 'gene_id' in i]
                gene_name = [i for i in attribute if 'gene_name' in i]
                gene_type = [i for i in attribute if 'gene_type' in i]

                if len(gene_id) > 0 and len(gene_name) > 0 and len(gene_type) > 0:
                    gene_id = gene_id[0].replace('gene_id ', '').replace('"', '').strip()
                    gene_id = re.sub('\\.[0-9]*', '', gene_id)
                    gene_name = gene_name[0].replace('gene_name ', '').replace('"', '').strip()
                    gene_type = gene_type[0].replace('gene_type ', '').replace('"', '').strip()
                    temp_gene = [gene_id, gene_name, gene_type]
                    # print(temp_gene)
                    gene_info.append(temp_gene)

gene_info = pd.DataFrame(gene_info)
gene_info.columns = ["gene_id", "gene_name", "gene_type"]

gene_info.to_csv("GTF_file.tsv", sep = "\t", index=False)

In [36]:
df = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                 'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})

In [37]:
df

,key,A
0,K0,A0
1,K1,A1
2,K2,A2
3,K3,A3
4,K4,A4
5,K5,A5


In [38]:
other = pd.DataFrame({'key': ['K0', 'K1', 'K2'],
                       'B': ['B0', 'B1', 'B2']})

In [39]:
other

,key,B
0,K0,B0
1,K1,B1
2,K2,B2


In [42]:
df.join(other, lsuffix='_caller', rsuffix='_other')

,key_caller,A,key_other,B
0,K0,A0,K0,B0
1,K1,A1,K1,B1
2,K2,A2,K2,B2
3,K3,A3,NaN,NaN
4,K4,A4,NaN,NaN
5,K5,A5,NaN,NaN
